In [1]:
#!/lustre7/home/lustre4/ryoyokosaka/python/.pyenv/shims
import sys
sys.path.append('/lustre7/home/lustre4/ryoyokosaka/python/')

In [2]:
import pandas as pd
import numpy as np
import glob
import re

In [9]:
path = glob.glob("/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/*_probe.txt")
path

['/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008364_93219_7326_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008385_93252_7342_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008380_93249_7339_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008398_93264_7354_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008368_93223_7330_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008372_93227_7334_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008325_93275_7365_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008316_93180_7287_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008320_93183_7290_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008394_93260_7350_probe.txt',
 '/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008334_93193_7300_probe.txt',

In [10]:
#かく転写産物が一回ずつ現れるように前処理する。
#SYMBOL行で重複のある転写産物名のリストをえる。
data_ID_exist = pd.read_table("/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008309_93173_7280_probe.txt",index_col="PROBE_ID")
SYMBOL = data_ID_exist["SYMBOL"]
SYMBOL_bool = SYMBOL.duplicated()
SYMBOL_duplicated = SYMBOL_bool[SYMBOL_bool == True]

In [11]:
#重複プローブを持つ転写物名リストを取得。
duplicated_transcript = list(SYMBOL[SYMBOL_duplicated.index].unique())

In [ ]:
GSE41080_expression_df = pd.DataFrame()
for txt_file in path:
    print(txt_file)
    data = pd.read_table(txt_file,index_col="PROBE_ID")
    signal_name = re.findall('\w\w\w\w\w\w\w\w\w\w_\w.AVG_Signal',str(data.columns))[0]
    data = data.rename(columns = {signal_name:'Signal'})
    data = pd.concat([data,SYMBOL],axis = 1)
    #ファイルパスからサンプルめい(GSM...を取得する)
    pattern = 'GSM\w\w\w\w\w\w\w'
    sample_name = re.findall(pattern,txt_file)


    #重複プローブの平均値を計算し新しいデータフレームに格納
    #新しいデータフレームを用意
    _expression_df = pd.DataFrame()
    list_read_SYMBOL = []

    for symbol in list(data['SYMBOL']):
        #もしプローブidが重複プローブであるならば平均値を計算する
        if symbol in duplicated_transcript:
            mean = data.loc[data.SYMBOL == symbol,'Signal'].mean()
            low = pd.DataFrame(mean,index=[symbol],columns=['Signal'])
            _expression_df = pd.concat([_expression_df,low])
        #もしプローブidが重複していないのであればシグナルはそのまま
        elif symbol not in list_read_SYMBOL:
            #今参照しているインデックス名をindに入れる
            ind = list(data[data["SYMBOL"] == symbol].index)
            low = pd.DataFrame(float(data['Signal'].loc[ind]),index=[symbol],columns=['Signal'])
            _expression_df = pd.concat([_expression_df,low])
        #すでに重複シンボルに対して平均値を計算していた場合そのループを飛ばす。
        elif symbol in list_read_SYMBOL:
            continue



        #読んだシンボルリストに追加
        list_read_SYMBOL.append(symbol)
    _expression_df = _expression_df.rename(columns = {"Signal":"sample_name"})
    GSE41080_expression_df = pd.concat([GSE41080_expression_df,_expression_df],axis=1)

/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008364_93219_7326_probe.txt
/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSM1008385_93252_7342_probe.txt


In [7]:
GSE41080_expression_df

""


In [ ]:
GSE41080_expression_df.to_csv("/lustre7/home/lustre4/ryoyokosaka/python/data/GSE41080/GSE41080_expression.csv")